In [8]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import twokenize
from nltk.tokenize import TweetTokenizer
from FeatureFunctions import getfeatures
from FeatureFunctions import helper

import sklearn
import csv


### Import the classifier
import sys
sys.path.insert(0, 'libsvm')

from svmutil import *

In [2]:
# whichtask = input("Which task to you want to do (A/B): ")
whichtask = "A"

In [13]:
### Reading train and test files

datafile = "datasets/train/SemEval2018-T3-train-task"+whichtask+"_emoji.txt"
# datafile2 = "datasets/train/SemEval2018-T3-train-taskA_emoji_ironyHashtags.txt"
trainingdata = pd.read_csv(datafile, delimiter = "\t",  quoting=csv.QUOTE_NONE, header=0)
trainingdata = trainingdata[['Label','Tweet text']]

testfile = 'datasets/goldtest_Task'+whichtask+'/SemEval2018-T3_gold_test_task'+whichtask+'_emoji.txt'

testdata = pd.read_csv(testfile, sep="\t",  quoting=csv.QUOTE_NONE, header=0)
testdata = testdata[['Label','Tweet text']]

testfileB = 'datasets/goldtest_TaskB/SemEval2018-T3_gold_test_taskB_emoji.txt'
testdataB = pd.read_csv(testfileB, sep="\t",  quoting=csv.QUOTE_NONE, header=0)
testdataB = testdataB[['Label','Tweet text']]

In [4]:
### Get lexical features
# training_features
lexical_training_features, unicount_vect, bicount_vect, tricount_vect, fourcount_vect = getfeatures.getlexical(trainingdata, 'Tweet text')
x_small = lexical_training_features[['PunctuationFlood', 'CharFlood', 'CapitalizedCount', 'HashtagCount', 'Hashtag2WordRatio', 'TweetCharLength', 'TweetWordLength', 'EmojiCount', 'FinalPunctuation']].values
x_lexical = np.array(lexical_training_features.apply(lambda row: sum([row['CharFourgramVector'], row['CharTrigramVector'],row['BigramVector']], []), axis=1).values.tolist())
x_lexical = np.hstack((x_small, x_lexical))

# train_bow = lexical_training_features['UnigramVector'].values.tolist()

# test_features
lexical_test_features, unicount_vect, bicount_vect, tricount_vect, fourcount_vect = getfeatures.getlexical(testdata, 'Tweet text', unicount_vect, bicount_vect, tricount_vect, fourcount_vect)
test_x_small = lexical_test_features[['PunctuationFlood', 'CharFlood', 'CapitalizedCount', 'HashtagCount', 'Hashtag2WordRatio', 'TweetCharLength', 'TweetWordLength', 'EmojiCount', 'FinalPunctuation']].values
test_lexical_x = np.array(lexical_test_features.apply(lambda row: sum([row['CharFourgramVector'], row['CharTrigramVector'],row['BigramVector']], []), axis=1).values.tolist())
test_lexical_x = np.hstack((test_x_small, test_lexical_x))

train_bow = np.array(lexical_training_features['UnigramVector'].values.tolist())
test_bow = np.array(lexical_test_features['UnigramVector'].values.tolist())

lexical_training_features = []
lexical_test_features = []




In [5]:
from importlib import reload  # Python 3.4+ only.
reload(getfeatures)

### Get sentiment features
train_sentiment_x = getfeatures.getaffinfeats(trainingdata['Tweet text'])
test_sentiment_x = getfeatures.getaffinfeats(testdata['Tweet text'])


In [6]:
def getindices(trainingdata, testdata):
    ### Creating 50-50% balance
    ## Train

    ### Count the amount of samples
    amount_nonirony_train = sum(trainingdata["Label"] == 0)
    amount_irony_train = sum(trainingdata["Label"] > 0)
    amount_train_amount = min(amount_nonirony_train, amount_irony_train)

    ### Sample indices
    train_nonirony_index = trainingdata[trainingdata["Label"] == 0].index.to_series()
    train_irony_index = trainingdata[trainingdata["Label"] > 0].index.to_series()

    train_nonirony_index_samples = train_nonirony_index.sample(amount_train_amount).tolist()
    train_irony_index_samples = train_irony_index.sample(amount_train_amount).tolist()

    ## Test
    amount_nonirony_test = sum(testdata["Label"] == 0)
    amount_irony_test = sum(testdata["Label"] > 0)
    amount_test_amount = min(amount_nonirony_test, amount_irony_test)

    ### Sample indices
    test_nonirony_index = testdata[testdata["Label"] == 0].index.to_series()
    test_irony_index = testdata[testdata["Label"] > 0].index.to_series()

    test_nonirony_index_samples = test_nonirony_index.sample(amount_test_amount).tolist()
    test_irony_index_samples = test_irony_index.sample(amount_test_amount).tolist()
    
    resulting_train_index = train_nonirony_index_samples +train_irony_index_samples
    resulting_test_index = test_nonirony_index_samples +test_irony_index_samples

    return resulting_train_index, resulting_test_index


In [10]:
### Combining features
iterations = int(input("How many iterations: "))
whichtype = int(input("Which type to test: "))
results = []

for i in range(iterations):
    print()
    print("-------------------------------------------------------------------")
    print("Iteration " + str(i+1) + "/" + str(iterations))
    print("-------------------------------------------------------------------")
    print()
    
    ### Selecting the actual training samples (Do some crossover here )
    resulting_train_index, resulting_test_index = getindices(trainingdata, testdata)
    trainingdata_result = trainingdata.loc[resulting_train_index]
    testdata_result = testdata.loc[resulting_test_index]
    
    final_train_x = []
    final_test_x = []

    ## BoW
    if whichtype == 1:
        final_train_x = train_bow[resulting_train_index]
        final_test_x = test_bow[resulting_test_index]
    ## Lexical
    if whichtype == 2:
        final_train_x = np.hstack((train_bow[resulting_train_index], x_lexical[resulting_train_index]))
        final_test_x = np.hstack((test_bow[resulting_test_index], test_lexical_x[resulting_test_index]))
    ## Sentiment
    if whichtype == 4:
        final_train_x = train_sentiment_x[resulting_train_index]
        final_test_x = test_sentiment_x[resulting_test_index]
    ## Combined
    if whichtype == 6:
        final_train_x = np.hstack((train_bow[resulting_train_index], train_sentiment_x[resulting_train_index], x_lexical[resulting_train_index]))
        final_test_x = np.hstack((test_bow[resulting_test_index], test_sentiment_x[resulting_test_index], test_lexical_x[resulting_test_index]))


    print("###########")
    print("Training:")
    print(final_train_x.shape)
    print()
    ### Train and get train error
    y = trainingdata_result['Label'].tolist()
    prob  = svm_problem(y, final_train_x)
    param = svm_parameter('-t 2 -c 8 -g ' + str(2**-11))
    m = svm_train(prob, param)
    p_label, p_acc, p_val = svm_predict(y, final_train_x, m)
    ACC, MSE, SCC = evaluations(y, p_label)

    print()
    print("###########")
    print("Testing:")
    print(final_test_x.shape)
    print()


    ### Get the test
    test_y = testdata_result['Label'].tolist()

    test_p_label, test_p_acc, test_p_val = svm_predict(test_y, final_test_x, m)
    test_ACC, test_MSE, test_SCC = evaluations(test_y, test_p_label)
    print(sklearn.metrics.classification_report(test_y, test_p_label, digits=4))

    if whichtask == "A":
        p, r, f = helper.precision_recall_fscore(test_y, test_p_label, beta=1, labels=[0,1], pos_label=1)
    elif whichtask == "B":
        p, r, f = helper.precision_recall_fscore(test_y, test_p_label, beta=1, labels=[0,1,2,3])

    print("Precision: " + str(p))
    print("Recall: " + str(r))
    print("F1-score: " + str(f))
    
    results.append([test_ACC, p*100, r*100, f*100])

    y_actu = pd.Series(test_y, name='Actual')
    y_pred = pd.Series(test_p_label, name='Predicted')
    df_confusion = pd.crosstab(y_actu, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

    print()
    print()
    print(df_confusion)

How many iterations: 10
Which type to test: 4

-------------------------------------------------------------------
Iteration 1/10
-------------------------------------------------------------------

###########
Training:
(3822, 6)

Accuracy = 58.8435% (2249/3822) (classification)

###########
Testing:
(622, 6)

Accuracy = 54.3408% (338/622) (classification)
              precision    recall  f1-score   support

           0     0.5364    0.6399    0.5836       311
           1     0.5538    0.4469    0.4947       311

   micro avg     0.5434    0.5434    0.5434       622
   macro avg     0.5451    0.5434    0.5391       622
weighted avg     0.5451    0.5434    0.5391       622

Precision: 0.5537848605577689
Recall: 0.44694533762057875
F1-score: 0.494661921708185


Predicted  0.0  1.0  All
Actual                  
0          199  112  311
1          172  139  311
All        371  251  622

-------------------------------------------------------------------
Iteration 2/10
----------------

In [11]:
mean_results = np.mean(np.array(results), axis=0)
std_results = 2*np.std(np.array(results), axis=0)

if whichtype == 1:
    print("BoW results:")
elif whichtype == 2:
    print("Lexical results:")
elif whichtype == 4:
    print("Semantic results:")
elif whichtype == 6:
    print("Combined results:")
        

for result_i in range(len(mean_results)):
    sys.stdout.write("{:.2f}".format(mean_results[result_i]) + "±" + "{:.2f}".format(std_results[result_i]) + "\t")
print()

Semantic results:
54.04±1.60	54.98±2.19	44.69±0.50	49.30±1.00	


In [41]:
### Evaluate results per category (Sectie 5.2)

## Getting category information
used_testB = testdataB.loc[resulting_test_index]

## Nonirony
print("--- Non-ironic")
predB_nonirony = sum((used_testB['Label'] == 0) & (y_pred == 0))
testB_nonirony = sum((used_testB['Label'] == 0))
print(str(predB_nonirony) + "/" + str(testB_nonirony))
print("{:.2f}%".format(predB_nonirony/testB_nonirony * 100))
print()

## 1
print("--- 1")
predB_nonirony = sum((used_testB['Label'] == 1) & (y_pred == 1))
testB_nonirony = sum((used_testB['Label'] == 1))
print(str(predB_nonirony) + "/" + str(testB_nonirony))
print("{:.2f}%".format(predB_nonirony/testB_nonirony * 100))
print()

## 2
print("--- 2")
predB_nonirony = sum((used_testB['Label'] == 2) & (y_pred == 1))
testB_nonirony = sum((used_testB['Label'] == 2))
print(str(predB_nonirony) + "/" + str(testB_nonirony))
print("{:.2f}%".format(predB_nonirony/testB_nonirony * 100))
print()

## 3
print("--- 3")
predB_nonirony = sum((used_testB['Label'] == 3) & (y_pred == 1))
testB_nonirony = sum((used_testB['Label'] == 3))
print(str(predB_nonirony) + "/" + str(testB_nonirony))
print("{:.2f}%".format(predB_nonirony/testB_nonirony * 100))
print()


--- Non-ironic
145/311
46.62%

--- 1
52/164
31.71%

--- 2
28/85
32.94%

--- 3
15/62
24.19%

